In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
train_url = '/home/thinker/NLP/NLP/FromScratch/error detection/data/data/train_data.csv'
test_url = '/home/thinker/NLP/NLP/FromScratch/error detection/data/data/test_data.csv'
df_train = pd.read_csv(train_url)
df_test = pd.read_csv(test_url)
stop_words_df = pd.read_excel('/home/thinker/NLP/NLP/FromScratch/error detection/data/stopwords_bangla.xlsx',index_col=False)

In [2]:
STOPWORDS = set([word.strip() for word in stop_words_df['words']])

In [3]:
import re
def preprocess(x):
    html_pattern = re.compile('<.*?>')
    x = html_pattern.sub(r'', x)
    x = " ".join([word for word in str(x).split() if word not in STOPWORDS])
    return x
df_train['Comment'] = df_train['Comment'].apply(lambda x: preprocess(x))
df_test['Comment'] = df_test['Comment'].apply(lambda x:preprocess(x))

In [4]:
def co(x):
    d = ['Code Switching','Spelling','Multiple Errors','Grammatical']
    s = set(d)
    if x not in s:
        return "Correct"
    else:
        return x
df_train['Category'] = df_train['Category'].apply(lambda x:co(x))
df_test['Category'] = df_test['Category'].apply(lambda x:co(x))

In [5]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(df_train.Category)
Test_Y = Encoder.fit_transform(df_test.Category)

In [6]:
df_all  = pd.concat([df_train, df_test], ignore_index=True)
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(df_all['Comment'])
Train_X_Tfidf = Tfidf_vect.transform(df_train['Comment'])
Test_X_Tfidf = Tfidf_vect.transform(df_test['Comment'])

In [7]:
param_grid = {'alpha': [0.01, 0.1, 1.0]}
grid_search = GridSearchCV(estimator=naive_bayes.MultinomialNB(), param_grid=param_grid, scoring='accuracy', cv=5)
grid_search.fit(Train_X_Tfidf, Train_Y)
best_nb_model = grid_search.best_estimator_

In [8]:
def evaluate_model(model, test_data, test_labels):
    predictions = model.predict(test_data)
    report = classification_report(test_labels, predictions, digits=4)
    print(report)

In [9]:
print("Naive Bayes:")
evaluate_model(best_nb_model, Test_X_Tfidf, Test_Y)

Naive Bayes:
              precision    recall  f1-score   support

           0     0.7000    0.2658    0.3853       158
           1     0.6129    0.9196    0.7356      1157
           2     0.1053    0.0156    0.0272       128
           3     0.1429    0.0145    0.0263        69
           4     0.4497    0.1703    0.2471       499

    accuracy                         0.5937      2011
   macro avg     0.4022    0.2772    0.2843      2011
weighted avg     0.5308    0.5937    0.5174      2011

